In [1]:
# importa todo y define una función de ejecutar pruebas
from importlib import reload
from ayudante import ejecutar_prueba

import cce
import ayudante
import script
import tx

### Ejercicio 1

#### 1.1. Haz que pase [esta prueba](/edit/sesión5/tx.py)
```
tx.py:PruebaTx::prueba_verificar_input
```

In [2]:
# Ejercicio 1.1

reload(tx)
ejecutar_prueba(tx.PruebaTx('prueba_verificar_input'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.265s

OK


In [3]:
# Ejemplo de construcción de transacción

from cce import ClavePrivada
from ayudante import decodificar_base58, SIGHASH_ALL
from script import script_p2pkh, Script
from tx import TxIn, TxOut, Tx

# Paso 1
tx_ins = []
tx_previa = bytes.fromhex('8be2f69037de71e3bc856a6627ed3e222a7a2d0ce81daeeb54a3aea8db274149')
indice_previo = 4
tx_ins.append(TxIn(
            tx_previa=tx_previa,
            indice_previo=indice_previo,
            script_sig=Script([]),
            sequence=0xffffffff,
        ))

# Paso 2
tx_outs = []
h160 = decodificar_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    cantidad=int(0.38*100000000),
    script_pubkey=script_p2pkh(h160),
))
h160 = decodificar_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    cantidad=int(0.1*100000000),
    script_pubkey=script_p2pkh(h160),
))
tx_obj = Tx(versión=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# Paso 3
tipo_hash = SIGHASH_ALL
z = tx_obj.sig_hash(0, tipo_hash)
pk = ClavePrivada(secreto=8675309)
der = pk.firmar(z).der()
firma = der + tipo_hash.to_bytes(1, 'big')
sec = pk.punto.sec()
tx_obj.tx_ins[0].script_sig = Script([firma, sec])
print(tx_obj.serializar().hex())

0100000001494127dba8aea354ebae1de80c2d7a2a223eed27666a85bce371de3790f6e28b040000006a4730440220391b819284a511f408e2935de5437f97f8c32cbebd6ca37233024451e16e189c02203c566e48ac4f04d07e3401d09e32205566736227c44cf6d4e031390fa21ca9cc012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff0280d54302000000001d017601a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f018801ac80969800000000001d017601a914507b27411ccf7f16f10297de6cef3f291623eddf018801ac00000000


### Ejercicio 2

#### 2.1. Haz que pase [esta prueba](/edit/sesión5/tx.py)
```
tx.py:PruebaTx:prueba_firmar_input
```

In [4]:
# Ejercicio 2.1

reload(tx)
ejecutar_prueba(tx.PruebaTx('prueba_firmar_input'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.330s

OK


### Ejercicio 3

#### 3.1. Envia 0.04 TBTC a esta dirección 

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`

#### Ve aquí para enviar tu transacción: https://testnet.blockchain.info/pushtx

#### Bonus. Obtén unas monedas testnet y gasta ambos outputs (uno de tu dirección de cambio y otro desde el grifo/faucet testnet) a 

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`

#### Puedes obtener algunas monedas testnet en: https://testnet.coinfaucet.eu/en/

In [5]:
# Ejercicio 3.1

from cce import ClavePrivada
from ayudante import decodificar_base58, SIGHASH_ALL
from script import script_p2pkh, Script
from tx import TxIn, TxOut, Tx

tx_previa = bytes.fromhex('eb581753a4dbd6befeaaaa28a6f4576698ba13a07c03da693a65bce11cf9887a')
indice_previo = 1
dire_destino = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
cantidad_destino = 0.04
dire_cambio = 'mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2'
cantidad_cambio = 0.317
secreto = 8675309
priv = ClavePrivada(secreto=secreto)

# inicializa inputs
tx_ins = []
# crea un nuevo tx input con tx_previa, indice_previo, script_sig en blanco y max sequence
tx_ins.append(TxIn(
            tx_previa=tx_previa,
            indice_previo=indice_previo,
            script_sig=Script([]),
            sequence=0xffffffff,
        ))

# inicializa outputs
tx_outs = []
# decodifica el hash160 desde la dirección de destino
h160 = decodificar_base58(dire_destino)
# convierte hash160 a script p2pkh
script_pubkey = script_p2pkh(h160)
# convierte cantidad destino a satoshis (multiplica por 100 millones)
satoshis_destino = int(cantidad_destino*100000000)
# crea un nuevo tx output para destino con cantidad y script_pubkey
tx_outs.append(TxOut(
    cantidad=satoshis_destino,
    script_pubkey=script_pubkey,
))
# decodifica el hash160 desde la dirección de cambio
h160 = decodificar_base58(dire_cambio)
# convierte hash160 a script p2pkh
script_pubkey = script_p2pkh(h160)
# convierte cantidad cambio a satoshis (multiplica por 100 millones)
satoshis_cambio = int(cantidad_cambio*100000000)
# crea un nuevo tx output para destino con cantidad y script_pubkey
tx_outs.append(TxOut(
    cantidad=satoshis_cambio,
    script_pubkey=script_pubkey,
))

# crea la transacción
tx_obj = Tx(versión=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# ahora firma el input 0 con la clave privada usando SIGHASH_ALL y firmar_input
tx_obj.firmar_input(0, priv, SIGHASH_ALL)

# COMPROBACIÓN: dirección de cambio corresponde a la clave privada
if priv.punto.dire(testnet=True) != dire_cambio:
    raise RuntimeError('La Clave Privada no corresponde a la dirección de cambio, comprueba priv_key y dire_cambio')

# COMPROBACIÓN: el script_pubkey del output es el mismo que tu dirección
if tx_ins[0].script_pubkey(testnet=True).cosas[2] != decodificar_base58(dire_cambio):
    raise RuntimeError('No te puedes gastar este output con esta clave privada. Comprueba que la tx_previa e indice_previo sean correctos')

# COMPROBACIÓN: la comisión es razonable
if tx_obj.comisión() > 0.05*100000000 or tx_obj.comisión() <= 0:
    raise RuntimeError('Comprueba que la cantidad de cambio sea razonable. La comisión es {}'.format(tx_obj.comisión()))

# serializar y hex()
print(tx_obj.serializar().hex())

01000000017a88f91ce1bc653a69da037ca013ba986657f4a628aaaafebed6dba4531758eb010000006a47304402201e018d85765dc32a28b7c403c151823bf683c71d5c9b5e47151fd76869c5154a02204ccc9531d7bacd5884b4ab9eb45b3540f35b5f7e3b42307ddfcd8911db943ab6012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff0200093d00000000001d017601a914ad346f8eb57dee9a37981716e498120ae80e44f7018801ac20b4e301000000001d017601a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f018801ac00000000


In [6]:
# Bonus

from cce import ClavePrivada
from ayudante import decodificar_base58, SIGHASH_ALL
from script import script_p2pkh, Script
from tx import TxIn, TxOut, Tx

tx_previa_1 = bytes.fromhex('89cbfe2eddaddf1eb11f5c4adf6adaa9bca4adc01b2a3d03f8dd36125c068af4')
indice_previo_1 = 0
tx_previa_2 = bytes.fromhex('19069e1304d95f70e03311d9d58ee821e0978e83ecfc47a30af7cd10fca55cf4')
indice_previo_2 = 0
dire_destino = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
cantidad_destino = 1.71
secreto = 61740721216174072121
priv = ClavePrivada(secreto=secreto)

# inicializa inputs
tx_ins = []
# crea el primer tx input con tx_previa_1, indice_previo_1, script_sig en blanco y max sequence
tx_ins.append(TxIn(
    tx_previa=tx_previa_1,
    indice_previo=indice_previo_1,
    script_sig=Script([]),
    sequence=0xffffffff,
))
# crea el segundo tx input con tx_previa_2, indice_previo_2, script_sig en blanco y max sequence
tx_ins.append(TxIn(
    tx_previa=tx_previa_2,
    indice_previo=indice_previo_2,
    script_sig=Script([]),
    sequence=0xffffffff,
))

# inicializa outputs
tx_outs = []
# decodifica el hash160 desde la dirección de destino
h160 = decodificar_base58(dire_destino)
# convierte hash160 a script p2pkh
script_pubkey = script_p2pkh(h160)
# convierte cantidad destino a satoshis (multiplica por 100 millones)
satoshis_destino = int(cantidad_destino*100000000)
# crea un único tx output para destino con cantidad y script_pubkey
tx_outs.append(TxOut(
    cantidad=satoshis_destino,
    script_pubkey=script_pubkey,
))

# crea la transacción
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# firma ambos inputs con la clave privada usando SIGHASH_ALL y firmar_input
tx_obj.firmar_input(0, priv, SIGHASH_ALL)
tx_obj.firmar_input(1, priv, SIGHASH_ALL)

# COMPROBACIÓN: script_pubkey del output es el mismo que tu dirección
if tx_ins[0].script_pubkey(testnet=True).cosas[2] != decodificar_base58(priv.punto.dire(testnet=True)):
    raise RuntimeError('El output no es algo que te puedas gastar con esta clave privada. Comprueba que tx_previa e indice_previo sean correctos')

# COMPROBACIÓN: la comisión es razonable
if tx_obj.comisión() > 0.05*100000000 or tx_obj.comisión() <= 0:
    raise RuntimeError('Comprueba que la cantidad de cambio sea razonable. La comisión es {}'.format(tx_obj.comisión()))

# serializa y hex()
print(tx_obj.serializar().hex())

0100000002f48a065c1236ddf8033d2a1bc0ada4bca9da6adf4a5c1fb11edfaddd2efecb89000000006b4830450221009bee08dddd8eec50589bfe5039dd35e3c31c966150ce760a60befc9cb9749c7d022039a4d02e45614297ec459646bb0d78bbb66ed1b8ef97961300740cda2f281846012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55befffffffff45ca5fc10cdf70aa347fcec838e97e021e88ed5d91133e0705fd904139e0619000000006b483045022100fe96c82abfa914581f463121181284f27f8dc8ca9f049b3c224a52179f78666602207a10c7566d2bfc74e9d29531ced8917f28bf6ad770b310e4f70c087f39baa587012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff01c040310a000000001d017601a914ad346f8eb57dee9a37981716e498120ae80e44f7018801ac00000000


### Ejercicio 4

#### 4.1. Encuentra el hash160 del RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [7]:
# Ejercicio 4.1

from ayudante import hash160

hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'

# bytes.fromhex script
redeem_script = bytes.fromhex(hex_redeem_script)
# hash160 result
h160 = hash160(redeem_script)
# hex() para  mostrar
print(h160.hex())

74d691da1574e6b3c192ecfb52cc8984ee7b6c56


In [8]:
# Ejemplo de construcción de dirección P2SH

from ayudante import codificar_base58_checksum

print(codificar_base58_checksum(b'\x05'+bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


### Ejercicio 5

#### 5.1. Haz que pasen [estas pruebas](/edit/sesión5/ayudante.py)
```
ayudante.py:PruebaAyudante:prueba_dire_p2pkh
ayudante.py:PruebaAyudante:prueba_dire_p2sh
```

In [9]:
# Ejercicio 5.1

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_dire_p2pkh'))
ejecutar_prueba(ayudante.PruebaAyudante('prueba_dire_p2sh'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK
.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [10]:
# ejemplo z para p2sh

from ayudante import doble_sha256

sha = doble_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
                    
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [11]:
# ejemplo de verificación p2sh

from cce import PuntoS256, Firma
from ayudante import doble_sha256

sha = doble_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
punto = PuntoS256.parsear(bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
firma = Firma.parsear(bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937'))
print(punto.verificar(z, firma))

True


### Ejercicio 6

#### 6.1. Valida la segunda firma del primer input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

El pubkey sec de la segunda firma es:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

La firma der de la segunda firma es:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

El redeemScript es:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [12]:
# Ejercicio 6.1

from io import BytesIO
from cce import PuntoS256, Firma
from ayudante import doble_sha256, int_a_little_endian
from script import Script
from tx import Tx, SIGHASH_ALL

hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script_stream = BytesIO(bytes.fromhex(hex_redeem_script))

hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(bytes.fromhex(hex_tx))

# parsear the PuntoS256 and Firma
punto = PuntoS256.parsear(sec)
firma = Firma.parsear(der)
# parsear la Tx
t = Tx.parsear(stream)
# cambia el scriptSig del primer input a redeemScript
# usa Script.parsear en el redeem_script_stream
t.tx_ins[0].script_sig = Script.parsear(redeem_script_stream)
# obtén la serialización
ser = t.serializar()
# añade el sighash (4 bytes, little-endian de SIGHASH_ALL)
ser += int_a_little_endian(SIGHASH_ALL, 4)
# doble_sha256 el resultado
a_firmar = doble_sha256(ser)
# esto interpretado como un número big-endian es tu z
z = int.from_bytes(a_firmar, 'big')
# ahora verifica la firma usando punto.verificar
print(punto.verificar(z, firma))

True
